In [53]:
from pathlib import Path
from label_studio_sdk.client import LabelStudio
from fishsense_api_sdk.client import Client
import asyncio
from tqdm.notebook import tqdm
import json
from datetime import datetime

In [54]:
URL = "http://localhost:8000"

In [55]:
label_studio_api_key = Path("..") / ".label_studio_api_key"

label_studio_api_key.exists()

True

In [56]:
ls = LabelStudio(base_url=f"https://labeler.e4e.ucsd.edu", api_key=label_studio_api_key.read_text().strip())

In [57]:
async with Client(URL) as fs:
    dives = await fs.dives.get()

len(dives), dives[0]

(479,
 Dive(id=21, name='H slate dive 1', path='2023.08.03.FishSense.FSL-01D/H slate dive 1', dive_datetime=datetime.datetime(2023, 8, 3, 9, 11, 50, tzinfo=TzInfo(0)), priority=<Priority.LOW: 'LOW'>, flip_dive_slate=None, camera_id=1, dive_slate_id=None))

In [58]:
async with Client(URL) as fs:
    species_labels = await asyncio.gather(*[fs.labels.get_species_labels(dive.id) for dive in dives])
    species_labels = [label for sublist in species_labels for label in sublist]

len(species_labels), species_labels[0]

(1178,
 SpeciesLabel(id=1024, label_studio_task_id=222252, label_studio_project_id=57, image_url='https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/groups_jpeg/7252dfb0db0b855616e45607f58c1c93', updated_at=datetime.datetime(2025, 10, 20, 20, 42, 14, 832143, tzinfo=TzInfo(0)), completed=True, grouping=None, top_three_photos_of_group=None, slate_upside_down=True, laser_x=1970.159985665477, laser_y=1340.3491294496944, laser_label='Red Laser', content_of_image='Slate, Laser not on slate', fish_measurable_category=None, fish_angle_category=None, fish_curved_category=None, label_studio_json={'annotations': [{'id': 117942, 'result': [{'id': 'result1', 'type': 'keypointlabels', 'value': {'x': 49.08221189998697, 'y': 44.44128413294742, 'width': 0.2, 'keypointlabels': ['Red Laser']}, 'origin': 'prediction-changed', 'to_name': 'image', 'from_name': 'laser', 'image_rotation': 0, 'original_width': 4014, 'original_height': 3016}, {'id': 'm4K68yydiZ', 'type': 'taxonomy', 'value': {'taxonomy': [

In [59]:
label_studio_project_ids = list({label.label_studio_project_id for label in species_labels if label.label_studio_project_id is not None})

len(label_studio_project_ids), label_studio_project_ids

(7, [57, 58, 59, 60, 61, 62, 63])

In [60]:
tasks = []

for project_id in label_studio_project_ids:
    tasks += list(ls.tasks.list(project=project_id))

len(tasks), tasks[0]

(1178,
 LseTask(agreement=None, agreement_selected=None, annotations=[{'id': 117942, 'result': [{'id': 'result1', 'type': 'keypointlabels', 'value': {'x': 49.08221189998697, 'y': 44.44128413294742, 'width': 0.2, 'keypointlabels': ['Red Laser']}, 'origin': 'prediction-changed', 'to_name': 'image', 'from_name': 'laser', 'image_rotation': 0, 'original_width': 4014, 'original_height': 3016}, {'id': 'm4K68yydiZ', 'type': 'taxonomy', 'value': {'taxonomy': [['Slate', 'Laser not on slate']]}, 'origin': 'manual', 'to_name': 'image', 'from_name': 'species'}, {'id': 'V_e6mRtCE1', 'type': 'choices', 'value': {'choices': ['Slate upside down']}, 'origin': 'manual', 'to_name': 'image', 'from_name': 'slate'}], 'created_username': 'Christopher Crutchfield ccrutchf@ucsd.edu, 4', 'created_ago': '1\xa0month, 1\xa0week', 'completed_by': 4, 'was_cancelled': False, 'ground_truth': False, 'created_at': '2025-10-15T21:10:56.570698Z', 'updated_at': '2025-10-20T20:42:14.832143Z', 'draft_created_at': '2025-10-15T

In [61]:
label_studio_task_to_species_label_map = {
    label.label_studio_task_id: label
    for label in species_labels
}

len(label_studio_task_to_species_label_map)

1178

In [62]:
async with Client(URL) as fs:
    for task in tqdm(tasks):
        species_label = label_studio_task_to_species_label_map.get(task.id)

        email = task.annotations[0]["created_username"].split(",")[0].split(' ')[-1].strip()
        tqdm.write(f"Processing task {task.id} for user {email}")
        user = await fs.users.get(email=email)

        species_label.label_studio_json = json.loads(task.json())
        species_label.user_id = user.id
        species_label.updated_at = task.updated_at
        species_label.completed = task.is_labeled

        if len(task.annotations) > 0:
            species_label.updated_at = datetime.fromisoformat(task.annotations[0]["updated_at"])

            grouping = [r for r in task.annotations[0]["result"] if r["from_name"] == "grouping"]
            grouping = grouping[0]["value"]["choices"][0] if len(grouping) > 0 else None
            species_label.grouping = grouping

            top_three_photos_of_group = [r for r in task.annotations[0]["result"] if r["from_name"] == "exclude"]
            top_three_photos_of_group = top_three_photos_of_group[0]["value"]["choices"][0] == "Top 3 photos of group" if len(top_three_photos_of_group) > 0 else None
            species_label.top_three_photos_of_group = top_three_photos_of_group

            slate_upside_down = [r for r in task.annotations[0]["result"] if r["from_name"] == "slate"]
            slate_upside_down = slate_upside_down[0]["value"]["choices"][0] == "Slate upside down" if len(slate_upside_down) > 0 else None
            species_label.slate_upside_down = slate_upside_down

            laser_label_section = [r for r in task.annotations[0]["result"] if r["from_name"] == "laser"]
            laser_label_section = laser_label_section[0] if len(laser_label_section) > 0 else None

            if laser_label_section is not None:
                original_width = laser_label_section["original_width"]
                original_height = laser_label_section["original_height"]

                laser_x = laser_label_section["value"]["x"] * original_width / 100
                laser_y = laser_label_section["value"]["y"] * original_height / 100

                laser_label = laser_label_section["value"]["keypointlabels"][0]

                species_label.laser_x = laser_x
                species_label.laser_y = laser_y
                species_label.laser_label = laser_label

            content_of_image = [r for r in task.annotations[0]["result"] if r["from_name"] == "species"]
            content_of_image = ", ".join(content_of_image[0]["value"]["taxonomy"][0]) if len(content_of_image) > 0 else None
            species_label.content_of_image = content_of_image

            fish_measurable_category = [r for r in task.annotations[0]["result"] if r["from_name"] == "measurable"]
            fish_measurable_category = [r for r in task.annotations[0]["result"] if r["from_name"] == "measurable"][0]["value"]["taxonomy"][0][0] if len(fish_measurable_category) > 0 else None
            species_label.fish_measurable_category = fish_measurable_category

            fish_angle_category = [r for r in task.annotations[0]["result"] if r["from_name"] == "fishAngles"]
            fish_angle_category = [r for r in task.annotations[0]["result"] if r["from_name"] == "fishAngles"][0]["value"]["taxonomy"][0][0] if len(fish_angle_category) > 0 else None
            species_label.fish_angle_category = fish_angle_category

            fish_curved_category = [r for r in task.annotations[0]["result"] if r["from_name"] == "fishCurve"]
            fish_curved_category = [r for r in task.annotations[0]["result"] if r["from_name"] == "fishCurve"][0]["value"]["taxonomy"][0][0] if len(fish_curved_category) > 0 else None
            species_label.fish_curved_category = fish_curved_category 

        await fs.labels.put_species_label(species_label.image_id, species_label)

  0%|          | 0/1178 [00:00<?, ?it/s]

Processing task 222252 for user ccrutchf@ucsd.edu
Processing task 222253 for user ccrutchf@ucsd.edu
Processing task 222254 for user jen.loch@reef.org
Processing task 222255 for user jen.loch@reef.org
Processing task 222256 for user jen.loch@reef.org
Processing task 222257 for user jen.loch@reef.org
Processing task 222258 for user jen.loch@reef.org
Processing task 222259 for user jen.loch@reef.org
Processing task 222260 for user jen.loch@reef.org
Processing task 222261 for user jen.loch@reef.org
Processing task 222262 for user jen.loch@reef.org
Processing task 222263 for user jen.loch@reef.org
Processing task 222264 for user jen.loch@reef.org
Processing task 222265 for user jen.loch@reef.org
Processing task 222266 for user jen.loch@reef.org
Processing task 222267 for user jen.loch@reef.org
Processing task 222268 for user jen.loch@reef.org
Processing task 222269 for user jen.loch@reef.org
Processing task 222270 for user jen.loch@reef.org
Processing task 222271 for user jen.loch@reef.org


In [63]:
[r for r in task.annotations[0]["result"] if r["from_name"] == "fishCurve"][0]["value"]["taxonomy"][0][0]


'No Curve'